In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages

In [4]:
path = 'C:\\Users\\user\\Desktop\\'
title = 'Plot'
with PdfPages(path+title+'.pdf') as pdf:

    Units = df.index.unique(level='Unit')
    pages = len(Units)+1

    years = mdates.YearLocator()
    years_formatter = mdates.DateFormatter('%Y')
    months = mdates.MonthLocator()
    months_formatter = mdates.DateFormatter('%b')
    formatter = ticker.StrMethodFormatter('{x:,.0f}')

    color_bne = 'tab:red'
    color_pemb = 'yellowgreen'
    color_diff = 'tab:orange'
    color_cumdiff = 'tab:blue'
    alpha_bar = 0.4
    alpha_line = 0.7
    labelcolor = 'k'
    linewidth = 1
    line_secondary_style = (0,(5,3))
    line_tertiary_style = (0,(1,2))
    barwidth = 5
    grid_lw = 0.5
    grid_ls = (0,(5,5))
    grid_ls2 = (0,(1,10))
    legend_fs='x-small'
    
    def graph(df: pd.DataFrame, header: str):

        ts = df.index.get_level_values('Date')

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11.75, 8.25), sharex=True)
        fig.suptitle(header, y=0.97)
        
        d = {}
        for n, ax in zip(('1b','2b'),(ax1, ax2)):
            ax.grid(lw=grid_lw, axis='x', which='minor', ls=grid_ls)
            ax.grid(lw=grid_lw, axis='x', which='major')
            ax.grid(lw=grid_lw, axis='y', which='major')
            ax.yaxis.set_major_formatter(formatter)
            axb = f'ax{n}'
            d[axb] = ax.twinx()
            d[axb].grid(axis='y', lw=grid_lw, ls=grid_ls2)
            d[axb].yaxis.set_major_formatter(formatter)

            
        bne_line = ax1.plot(ts, df['a'], color=color_bne, label='a', linewidth=linewidth)
        pemb_line = ax1.plot(ts, df['b'], color=color_pemb, label='b', linewidth=linewidth, alpha=alpha_line)
        ax1.tick_params(axis='y', labelcolor=labelcolor)
        ax1.set_ylabel('m³/d', color=labelcolor)
        handles, labels = ax1.get_legend_handles_labels()
        ax1.legend(handles, labels, loc=2, fontsize=legend_fs)

        df_bar = df.groupby(pd.Grouper(level='Date', freq='MS')).sum()
        ts_bar = mdates.date2num(df_bar.index.shift(14, freq='D'))
        ts_bar_bne = ts_bar - barwidth/2
        ts_bar_pemb = ts_bar + barwidth/2
        bne_bar = d['ax1b'].bar(ts_bar_bne, df_bar['a'], color=color_bne, width=barwidth, alpha=alpha_bar)
        pemb_bar = d['ax1b'].bar(ts_bar_pemb, df_bar['b'], color=color_pemb, width=barwidth, alpha=alpha_bar)
        d['ax1b'].tick_params(axis='y', labelcolor=labelcolor)
        d['ax1b'].set_ylabel('m³/month', color=labelcolor)

        
        diff = ax2.plot(ts, df['diff'], color=color_diff, linewidth=linewidth, alpha=alpha_line)
        ax2.tick_params(axis='y', labelcolor=color_diff)
        ax2.set_ylabel('Difference (m³/d)', color=color_diff)

        cumdiff = d['ax2b'].plot(ts, df['diff'].fillna(0).cumsum(), color=color_cumdiff, linewidth=linewidth)
        if header in secondary_meters:
            seccumdiff =\
            d['ax2b'].plot(ts, df['Secondary diff'].fillna(0).cumsum(), color=color_cumdiff, label='Secondary Meter', linewidth=linewidth, ls=line_secondary_style)
            if header in tertiary_meters:
                d['ax2b'].plot(ts, df['Tertiary diff'].fillna(0).cumsum(), color=color_cumdiff, label='Tertiary Meter', linewidth=linewidth, ls=line_tertiary_style)
            d['ax2b'].legend(loc=2, fontsize=legend_fs)

        d['ax2b'].tick_params(axis='y', labelcolor=color_cumdiff)
        d['ax2b'].set_ylabel('Cumulative Difference (m³)', color=color_cumdiff)

        ax2.xaxis.set_major_locator(years)
        ax2.xaxis.set_major_formatter(years_formatter)
        ax2.xaxis.set_minor_locator(months)
        ax2.xaxis.set_minor_formatter(months_formatter)

        fig.subplots_adjust(left=0.1, bottom=0.07, right=0.91, top=0.94, hspace=0.03)
        fig.text(0.5, 0.02, str(pdf.get_pagecount()+1)+' of '+str(pages), size=8, ha='center')
        pdf.savefig(fig)
        plt.close(fig)
        fig.clf()
        
    graph(df.groupby(level='Date').sum(min_count=1), 'All Units')

    for Unit in Units:
        df_0 = df.xs(Unit, level='Unit')
        graph(df_0, Unit)